# Assignment Prompt - Retail Store Location Scraper


In [ ]:

#Importing required libraries
import requests
from bs4 import BeautifulSoup
import csv

# Here I have choosen the spencers as my favourite retail brand in India to extract the data 
url = 'http://www.spencersretail.com/store-list?page=2'

# Send a GET request to the specified URL and store the response in a variable called 'response'
response = requests.get(url)

# Parse the HTML content of the response using BeautifulSoup and store the resulting object in a variable called 'soup
soup = BeautifulSoup(response.text, 'html.parser')

# Find all the store names, addresses, postal codes, localities, regions, contact numbers, and opening/closing times using their respective HTML tags and attributes
store_name = soup.find_all("span", class_="fn")
store_address = soup.find_all("span", itemprop="streetAddress")
postal_code = soup.find_all("span", class_="postal-code")
locality = soup.find_all("span", class_="locality")
region = soup.find_all("span", class_="region")
Contact = soup.find_all("span", itemprop="telephone")
Time = soup.find_all("div", class_="field-content")

# Extract the opening times and closing times from the 'Time' variable and store them in separate lists called 'Opens_at' and 'Closes_at'
Opens_at=[]
for i in range(1,len(Time),3):
    Opens_at.append(Time[i])
    
Closes_at = []
for i in range(2,len(Time),3):
    Closes_at.append(Time[i])

# Create a list called 'full_address' containing dictionaries with the full address for each store, I will use this address list further to get the latitute and longitude
full_address=[]
for i in range(len(store_name)):
    full_address.append({
        store_address[i].text.strip() + postal_code[i].text.strip() + locality[i].text.strip() + region[i].text.strip()       
    })


# Now I will use an API from the wesite "Rapid API", which will give us the latitude and longitude of the addresses of the stores

# Set up the API endpoint URL and headers to access the TrueWay Geocoding API
url = "https://trueway-geocoding.p.rapidapi.com/Geocode"

headers = {
	"X-RapidAPI-Key": "b2dcd3ea06msh60d0300569d7b6dp1f52a6jsn31f7da53e46b",
	"X-RapidAPI-Host": "trueway-geocoding.p.rapidapi.com"
}


# Query the TrueWay Geocoding API for the latitude and longitude of each store using their full address and store the results in a list called 'lat_long'
lat_long = []
for i in range(len(full_address)):
    querystring = {"address":full_address[i],"language":"en"}
    response = requests.request("GET", url, headers=headers, params=querystring)
    lat_long.append((response.json()['results'][0]["location"]))
                  

# Create a list of dictionaries to store the extracted data for each store
data = []
for i in range(len(store_name)):
    data.append({
        "Store Name": store_name[i].text.strip(),
        "Store Address": store_address[i].text.strip(),
        "Postal Code": postal_code[i].text.strip(),
        "Locality": locality[i].text.strip(),
        "Region": region[i].text.strip(),
        "Contact": Contact[i].text.strip(),
        "Opens At": Opens_at[i].text.strip(),
        "Closes At": Closes_at[i].text.strip(),
        "Lat-Long": lat_long[i]
    })

# Write the data into a CSV file called 'spencers.csv'
with open('spencers.csv', mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=data[0].keys())
    writer.writeheader()
    for row in data:
        writer.writerow(row)
